### 1️⃣ Convert MXD Files to APRX Projects

In [ ]:
import arcpy
import os

mxd_folder = r"GIS\arcpy"
output_folder = r"ConvertedFiles"
template_aprx = r"base.aprx"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

arcpy.env.workspace = mxd_folder
mxds = arcpy.ListFiles("*.mxd")

for mxd_name in mxds:
    try:
        mxd_path = os.path.join(mxd_folder, mxd_name)
        aprx_name = os.path.splitext(mxd_name)[0] + ".aprx"
        aprx_path = os.path.join(output_folder, aprx_name)

        if os.path.exists(aprx_path):
            os.remove(aprx_path)
            print(f"[~] Replaced existing: {aprx_name}")

        # Copy blank base project
        aprx_template = arcpy.mp.ArcGISProject(template_aprx)
        aprx_template.saveACopy(aprx_path)

        # Open and import MXD
        aprx = arcpy.mp.ArcGISProject(aprx_path)
        aprx.importDocument(mxd_path)

        # Create writable native map
        new_map = aprx.createMap("FinalMap")

        # Copy layers from imported map
        for m in aprx.listMaps():
            if m.name != "FinalMap":
                for lyr in m.listLayers():
                    new_map.addLayer(lyr)
                break

        aprx.save()
        print(f"[✓] Converted {mxd_name} to {aprx_name}")

    except Exception as e:
        print(f"[X] Failed to convert {mxd_name}: {e}")


### 2️⃣ Repair Broken Layer Connections

In [ ]:
# Repair broken layers in APRX files
# Folder containing APRX files with FinalMap
aprx_folder = r"ConvertedFiles"
data_repair_folder = r"GIS\arcpy"

aprx_files = [f for f in os.listdir(aprx_folder) if f.endswith(".aprx")]

for aprx_name in aprx_files:
    try:
        aprx_path = os.path.join(aprx_folder, aprx_name)
        aprx = arcpy.mp.ArcGISProject(aprx_path)
        final_map = aprx.listMaps("FinalMap")[0]

        print(f"\n🔧 Checking: {aprx_name}")

        for lyr in final_map.listLayers():
            if lyr.isBroken:
                try:
                    for ext in [".shp", ""]:  # Extend this list for other formats
                        guess_path = os.path.join(data_repair_folder, lyr.name + ext)
                        if arcpy.Exists(guess_path):
                            lyr.updateConnectionProperties(None, guess_path)
                            print(f"    [+] Repaired: {lyr.name} → {guess_path}")
                            break
                    else:
                        print(f"    [!] Still broken: {lyr.name}")
                except Exception as e:
                    print(f"    [!] Error repairing {lyr.name}: {e}")

        aprx.save()
        print(f"[✓] Saved repaired: {aprx_name}")

    except Exception as e:
        print(f"[X] Failed to process {aprx_name}: {e}")
